In [1]:
import sys
sys.path.append("..")

# deeptrack - Tracking point particles

This notebook demonstrates how to track point particles.

## Setup

In [2]:
# Import all classes that are needed for this example
from deeptrack.scatterers import PointParticle
from deeptrack.optics import OpticalDevice
from deeptrack.image import Image

import numpy as np
import matplotlib.pyplot as plt

## Define the optics 

To image an object we need to define the properties of the optical system. This is done using an Optics instance. The role of the Optics instance is to take a number of light sources and correctly convolve them with the pupil function.

The optics is controlled by the following parameters:

* NA: The numerical aperature

* wavelength: The wavelength of the lightsource (m)

* pixel_size: The pixel to meter conversion factor (m/px)

* mode: "coherent" or "incoherent" light emitted by the object

* ROI: Region of interest that is imaged. Useful for avoiding wrap-around effects when doing Fourier tranforms

* upscale: How far to upscale the generated pupil-function. This increases output accuracy at the expense of additional comutational power.

In [ ]:
Optics = OpticalDevice(
    NA=0.7,                
    pixel_size=0.1e-6,     
    wavelength=680e-9
)

## Define the particle

The generator takes a set of features, which are rules for generating an image, and creates an Image instance.

In this case we consider a single feature, a particle. The particle object contains all the information needed to image an instance of that particle. 

A simple point particle, or a point source, is entirely defined by its intensity and its position.

In [ ]:
particle = PointParticle(                                         
    intensity=100,
    position=(32, 16),
    position_unit="pixel"
)

## Generate the image

Finally, we have everything needed to create an image! This is done using the class method `Generator.get`. This returns an Image object. This behaves exactly like a normal numpy ndarray, just additionally storing the properties used to generating the image as a list of dicts.

In [ ]:
output_image = Optics(particle).plot()

## Randomizing the particle

To teach a model to track a particle, it's position should be randomized. We do this by passing a lambda function to the `position` keyword. When this lambda function is called, it returns a random pair of numbers which represent the position. 

We also plot the position of the particle, to ensure that it's positioned correctly.

In [1]:
# Retrieves the current position of the particle
def get_position():
    return np.array(particle.properties["position"].current_value)

particle = PointParticle(                                         
    intensity=100,
    position=lambda: 10 + np.random.rand(2) * 44,
    position_unit="pixel"
)

input_image = Image(np.zeros((64, 64)))
output_image = Optics(particle).resolve(input_image)

position = get_position()
plt.gray()
plt.imshow(output_image)
plt.scatter(position[0], position[1])
plt.show()

NameError: name 'PointParticle' is not defined

In [ ]:
from deeptrack.generators import Generator
from deeptrack.models import convolutional


G = Generator()

model = deeptrackNetwork(input_shape=(64, 64, 1), number_of_outputs=2)

# Divide position by 64 to get value between 0 and 1
def get_position(image):
    return np.array(particle.properties["position"].current_value / 64)

generator = G.generate(Optics(particle), get_position, shape=(64, 64), batch_size=4)


model.fit(
    generator,
    epochs=1000,
    steps_per_epoch=64
)


In [ ]:
batch, label = next(generator)

prediction = model.predict(batch)

for i in range(batch.shape[0]):
    plt.gray()
    plt.imshow(np.squeeze(batch[i]))
    # Multiply back 64
    plt.scatter(prediction[i,0] * 64, prediction[i,1] * 64)
    plt.show()